### Initialize notebook

In [ ]:
# Torch imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import PIL.Image as IMG
from torch.utils.data.sampler import WeightedRandomSampler

# Other imports
import matplotlib.pyplot as plt
import numpy as np
import os

# NOTE: Aashis, you should use the training images to develop your algorithm.
# The test images should be reserved for validating its performance afterwards
# os.chdir('/home/ak/Spring2018/ature')
os.chdir('/home/ak/Spring2018/ature')
from utils import img_utils as imgutil

sep = os.sep

# Define folders (create them if needed)
Dirs = {}
Dirs['train_data']      = 'data'+sep+'DRIVE'+sep+'training'+sep +'patches'
Dirs['test_data']      = 'data'+sep+'DRIVE'+sep+'test'+sep +'patches'

# Set up execution flags
Flags = {}
Flags['useGPU'] = False

classes = ('white', 'green', 'black', 'red')

In [ ]:
class DriveDataset(Dataset):
    
    # data_path should contain pickled numpy array of dimension N * (D+1)
    # Where extra one dimension stores the correct lable among (0, 1, 2, 3)
    
    def __init__(self, data_path=None, height=None, width=None, transform=None):
        
        
        self.height = height
        self.width = width
        self.transform = transform
        
        self.data = None
        for data_file in os.listdir(data_path):
            
            data_file = os.path.join(data_path, data_file)
            print('Data file: ' + data_file)
            if self.data is None:
                self.data = np.load(data_file)
            else:
                self.data = np.concatenate((self.data, np.load(data_file)), axis=0)
            
    
        self.labels = torch.from_numpy(self.data[:, self.height * self.width]) 
        self.data = self.data[:,0:self.height * self.width]
        
    def __getitem__(self, index):
        
        img_arr = self.data[index].reshape(self.height, self.width)
        img = IMG.fromarray(imgutil.whiten_image2d(img_arr))
        
        if self.transform is not None:
            img_tensor = self.transform(img)
            
        return (img_tensor, self.labels[index])
    
    def __len__(self):
        return self.data.shape[0]

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])

trainset = DriveDataset(data_path=Dirs['train_data'], height=31, width=31, transform=transform)
clss, class_counts = np.unique(trainset.labels, return_counts=True)
class_weights = 1.0/class_counts
data_weights = np.array([class_weights[t] for t in trainset.labels]) 
second_min_class_count =  np.partition(class_counts, 1)[1]

sampler = WeightedRandomSampler(data_weights, int(second_min_class_count))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=False, num_workers=1, sampler=sampler)

In [ ]:
IMG.fromarray(trainset.data[3000].reshape(31,31))

### Define the network

In [ ]:
class Net(nn.Module):
    def __init__(self, width, channels):
        super(Net, self).__init__()
        
        self.channels = channels
        self.width = width
        
        
        self.kern_size = 5
        self.kern_stride = 2      
        self.kern_padding = 2
        
        self.mxp_kern_size = 2
        self.mxp_stride = 2 
        self.pool1 = nn.MaxPool2d(kernel_size=self.mxp_kern_size, stride=self.mxp_stride)
        
        self.conv1 = nn.Conv2d(self.channels, 20, self.kern_size, 
                               stride=self.kern_stride, padding=self.kern_padding)
        self._update_output_size()
        
        
        self.kern_size = 5
        self.kern_stride = 1      
        self.kern_padding = 1
        
        self.mxp_kern_size = 1
        self.mxp_stride = 1 
        self.pool2 = nn.MaxPool2d(kernel_size=self.mxp_kern_size, stride=self.mxp_stride)
        
        self.conv2 = nn.Conv2d(20, 50, self.kern_size, 
                               stride=self.kern_stride, padding=self.kern_padding)
        self._update_output_size()
        
        
        
        self.linearWidth = 50*int(self.width)*int(self.width)
        self.fc1 = nn.Linear(self.linearWidth, 30)
        self.fc2 = nn.Linear(30, 10)
        self.fc3 = nn.Linear(10, 4)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, self.linearWidth)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def _update_output_size(self):       
        self.width = ((self.width - self.kern_size + 2 * self.kern_padding) / self.kern_stride) + 1
        temp = self.width
        self.width = ((self.width - self.mxp_kern_size)/self.mxp_stride) + 1
        print('output width { conv: ' + str(temp) + ', maxpool: ' + str(self.width) + ' }')

width = 31
channels = 1
net = Net(width, channels)

# Send network to the GPU, if requested
if Flags['useGPU']:
    net.cuda()

# Define loss criterion
# criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the network

In [ ]:
for epoch in range(10): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        if Flags['useGPU']:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())                
        else:                
            inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics every 500 mini-batches
        running_loss += loss.data[0]
        if i % 1000 == 999:
            print('[epoch: %d, batches: %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

### Load test dataset

In [ ]:
testset = DriveDataset(data_path=Dirs['test_data'], height=31, width=31, transform=transform)
clss_test, class_counts_test = np.unique(testset.labels, return_counts=True)
class_weights_test = 1.0/class_counts_test

data_weights_test = np.array([class_weights_test[t] for t in testset.labels])
data_weights_test = np.ones_like(testset.labels)

second_min_class_count_test =  np.partition(class_counts_test, 1)[1]

sampler_test = WeightedRandomSampler(data_weights_test, int(second_min_class_count_test))
testloader = torch.utils.data.DataLoader(testset, batch_size=2, shuffle=False, num_workers=1, sampler=sampler_test)

### Test the network

In [ ]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
    
    images, labels = data  
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
    if i % 500 == 499:
        print('Accuracy of %d batches of test images: %d %%' % (i+1, 100 * correct / total))


### Check per-class performance

In [ ]:
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
i = 0
for data in testloader:
    
    images, labels = data
    
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(2):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
    i = i+1
    if i > 10000:
        break

for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
